## Contribution of Chidananda Pati/012506949
## It has code for 2 factors
- Spell Checking - More spelling error means mostly writeen by un-professionals and may be a fake news
- Political Affiliation - I initially thought of geography , but could not apply that to dataset

In [2]:
import numpy as np
import pandas as pd

In [3]:
columns=['id','label','statement','subject','speaker','speaker_job','state',
        'party_affiliation','barely_true_count','false_Count',
        'half_true_count','mostly_true_count','pants_on_fire_count','venue_speach'];
df_lair=pd.read_csv('../train.tsv',sep='\t',header=None,names=columns,index_col=False);
df_lair.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chidanandapati/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Text Proprocessing

#### Function for text preprocessing
- lowercase the text
- word tokenization
- remove stop words and non alphanumeric charaters
- stemming

In [5]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
def text_preprocessing(df_base,column):
    df=df_base.copy()
    # lowercase the text
    df[column]=df[column].str.lower()
    # word tokenization
    df[column]=df[column].map(lambda x: nltk.word_tokenize(x))
    # remove stop words and non alphanumeric charaters
    df[column]=df[column].map(lambda x: [w for w in x if (not w in stop_words) and w.isalpha()])
    # lemmatization
    wordnet_lemmatizer = WordNetLemmatizer()
    df[column]=df[column].map(lambda x: [ wordnet_lemmatizer.lemmatize(w) for w in x])    
    # stemming
    porter = PorterStemmer()
    df[column]=df[column].map(lambda x: [porter.stem(w) for w in x] )
    return df

#### Calling text_preprocessing function

In [6]:
df_train=text_preprocessing(df_lair,'statement')
df_train.head()

,id,label,statement,subject,speaker,speaker_job,state,party_affiliation,barely_true_count,false_Count,half_true_count,mostly_true_count,pants_on_fire_count,venue_speach
0,2635.json,false,"[say, anni, list, polit, group, support, abort...",abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,"[declin, coal, start, start, natur, ga, took, ...","energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"[hillari, clinton, agre, john, mccain, vote, g...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,"[health, care, reform, legisl, like, mandat, f...",health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,"[econom, turnaround, start, end, term]","economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


## Tag Cloud
To understand most frequent words in the corpus
- pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
statements=''.join(df_lair['statement'].values.tolist())

In [ ]:
# lower max_font_size, change the maximum number of word and lighten the background:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(statements)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Conclusion is corpus looks like is mostly political and money related

## Word2Vec to create a vector representation of processed words for every article
- pip install gensim

In [9]:
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [ ]:
def labelled_sentences(articles, label_type):
    labelledSentences = []
    for i,_d in enumerate(articles):
        labelledSentences.append(LabeledSentence(_d, label_type[i]))
    return labelledSentences

In [ ]:
df_train_labelled=labelled_sentences(df_train['statement'],df_train['label'])
df_train_labelled[1]

In [ ]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(df_train['statement'])]

In [ ]:
tagged_data_words=[x.words for x in tagged_data]
tagged_data_words_1D=[]
for row in range(len(tagged_data_words)):
    for col in range(len(tagged_data_words[row])):
        tagged_data_words_1D.append(tagged_data_words[row][col]) 

In [ ]:
n_dim=300
w2v = Word2Vec(size=n_dim, min_count=0)
w2v.build_vocab(tagged_data_words)
w2v.train(tagged_data_words,total_examples=w2v.corpus_count,epochs=w2v.epochs)

In [ ]:
w2v.most_similar('polit')

In [ ]:
df_train_statements=df_train[['statement','label']]
df_train_statements_vectorized=df_train_statements['statement'].map(lambda x: [w2v[w] for w in x])

In [ ]:
df_train_statements['statement']=df_train_statements_vectorized

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_train_statements['label']=le.fit_transform(df_train_statements['label'])

In [ ]:
df_train_statements.head()

In [ ]:
#list(le.inverse_transform(df_train_statements['label']))

In [ ]:
def document_vector(doc):
    return np.mean(doc, axis=0)

In [ ]:
for row_idx in range(len(df_train_statements['statement'])):
    for col_idx in range(len(df_train_statements['statement'][row_idx])):
        df_train_statements['statement'][row_idx][col_idx]=document_vector(df_train_statements['statement'][row_idx][col_idx])

In [ ]:
#df_train_statements['statement']=df_train_statements['statement'].map(document_vector)
df_train_statements.drop(index=4497,inplace=True)
df_train_statements.reset_index(drop=True)

##### Making word vec to features

In [ ]:
texts=[]
for x in df_train_statements['statement'].values:
    texts.append(x)   
X=pd.DataFrame(texts)
X=X.fillna(0)
y=df_train_statements[['label']]
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .2, random_state = 1)

In [ ]:
X_train.head()

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train, y_train)
gnb_pred = gnb.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test,gnb_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Lets try with 100 trees
num_trees=100
rf=RandomForestClassifier(n_estimators=num_trees)
rf.fit(X_train, y_train)
rf_pred=rf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,rf_pred))

## Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec

In [ ]:
df_train_statements_d2v=df_train[['statement','label']]

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model.all")
print("Model Saved")    

In [ ]:
model= Doc2Vec.load("d2v.model.all")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model.infer_vector(x))

In [ ]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')    

In [ ]:
X=pd.DataFrame(texts)
#y=df_train_statements_d2v[['label']]
y=df_train_statements_d2v['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [ ]:
np.unique(y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr_D2V = LogisticRegression(C=100)
logisticRegr_D2V.fit(X_train, y_train)
lr_pred = logisticRegr_D2V.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,lr_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Lets try with 100 trees
num_trees=100
rf=RandomForestClassifier(n_estimators=num_trees)
rf.fit(X_train, y_train)
rf_pred=rf.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,rf_pred))

In [ ]:
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

### K-means for vectorized docs

In [ ]:
from nltk.cluster import KMeansClusterer

In [ ]:
#Taking a very long time
#kclusterer = KMeansClusterer(5, distance=nltk.cluster.util.cosine_distance, repeats=5)
#assigned_clusters = kclusterer.cluster(texts, assign_clusters=True)
#assigned_clusters[0]

In [ ]:
#for i, word in enumerate(texts):  
#    print (texts[i] , assigned_clusters[i])
#print (texts[1] , assigned_clusters[1])

In [ ]:
from sklearn.cluster import KMeans as km
import matplotlib.pyplot as plt
def kmeans_plot(k):
    kmeans=km(n_clusters=k,random_state=0).fit(texts)
    labels=kmeans.labels_
    centroids=kmeans.cluster_centers_    
    colors = ['r','g','b','y','c','m','r','g','b','y','c','m','r','g','b','y','c','m']
    fig, ax=plt.subplots()
    
    for i in range(k):
        points=np.array([texts[j] for j in range(len(texts)) if labels[j]==i])
        ax.scatter(points[:,0],points[:,1],s=7,c=colors[i])
    ax.scatter(centroids[:,0],centroids[:,1],marker='*',s=200,c='#050505')     
kmeans_plot(5)    

## LDA Topic Modelling to see if we can find any topic

In [ ]:
from gensim import corpora, models
texts=df_train['statement']
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=20)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

### Topics Seems like are all related to Politics, broadly followings:
- Tax Cut
- Health Care
- Immigration and Tax Policies
- 2016 Presidential Campaigns

## Conclusion: Document with higher tf-idf has higher political affilication

## Kaggel Data set

In [ ]:
df_kgl=pd.read_csv('./kaggel/train.csv',header=0)
df_kgl.shape

In [ ]:
df_kgl.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sns.countplot(x="truth",data=df_kgl)

In [ ]:
import ast
#df_kgl[['subjects']].apply(lambda x:ast.literal_eval(x))
df_kgl['subjects_list']=df_kgl['subjects'].apply(lambda x:ast.literal_eval(x))

In [ ]:
subjects=[]
for col in df_kgl['subjects_list']:
    for val in col:
        subjects.append(val)

#### Top 20 Subjects

In [ ]:
from collections import Counter
letter_counts = Counter(subjects).most_common(20)
letter_counts
#df = pd.DataFrame.from_dict(letter_counts, orient='index')
#df.plot(kind='bar',figsize=(20,4))

In [ ]:
df_kgl_hc=df_kgl[df_kgl['subjects'].str.contains('Health Care')]
df_kgl_hc.head()

In [ ]:
df_kgl_hc_train=text_preprocessing(df_kgl_hc,'statement')
df_kgl_hc_train['statement'].head()

In [ ]:
sns.countplot(x="truth",data=df_kgl_hc_train)

In [ ]:
from gensim import corpora, models
texts=df_kgl_hc_train['statement']
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=20)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

## Factor Political Affiliation

In [ ]:
vocab_political_affiliation=['president','george','bush','administration','republican','democrats','barack','obama','hillary','clinton','donald','trump','senate','house']
#vocab_political_affiliation=['president','george','bush','administration']
#vocab_political_affiliation=['president','administration','republican','democrats','barack','clinton','donald','trump','senate','house']
len(vocab_political_affiliation)

In [ ]:
l_statments=df_lair['statement'].str.lower().values.tolist()
l_statments[0]

In [ ]:
l_tokenized_statements=df_train['statement'].map(lambda x:' '.join(x))
type(l_tokenized_statements)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=vocab_political_affiliation)
vectorizer

In [ ]:
def tf_idf_weights(tokenized_statements):
    tfidf_matrix = vectorizer.fit_transform(tokenized_statements)
    feature_names = vectorizer.get_feature_names()
    tf_idf_vals=[]
    for i in range(len(tfidf_matrix.toarray())):
        tf_idf_doc=np.zeros(len(vocab_political_affiliation))
        feature_index = tfidf_matrix[i,:].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[i,x] for x in feature_index])
        count=0
        for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            if (w in vocab_political_affiliation):
                tf_idf_doc[count]=s
                count+=1
        tf_idf_vals.append(tf_idf_doc.tolist())   
                #tf_idf_vals=tf_idf_vals[~(tf_idf_vals==0).all(1)]
                #tf_idf_vals[np.all(tf_idf_vals == 0,axis=1)]
    return np.array(tf_idf_vals)

In [ ]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')    

In [ ]:
X=tf_idf_weights(l_tokenized_statements)
y=df_lair['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .4, random_state = 1)
np.unique(y)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(C=100)
logisticRegr.fit(X_train, y_train)
lr_pred = logisticRegr.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test,lr_pred))

In [ ]:
def tokenize_statment(statement):
    statement=nltk.word_tokenize(statement.lower())
    statement=[w for w in statement if (not w in stop_words) and w.isalpha()]
    wordnet_lemmatizer = WordNetLemmatizer()
    statement=[ wordnet_lemmatizer.lemmatize(w) for w in statement]
    porter = PorterStemmer()
    statement=[porter.stem(w) for w in statement]
    return statement

In [ ]:
tokenize_statment("Says the Annies List political group supports ")

In [ ]:
l_statments[0]

In [ ]:
test_news="says the annies list political group supports third-trimester abortions on demand";
#statments=df_train['statement'].map(lambda x:' '.join(x))
#statments=statments.append(pd.Series(' '.join(tokenize_statment(test_news))))
#statments=statments.reset_index()
print(statments.iloc[-1])
#statments
tf_idf_weights(statments)[1]
#political_affiliation_checker(test_news)

## Doc2Vec for political affilication

In [7]:
vocab_political_affiliation=['president','george','bush','administration','republican','democrats','barack','obama','hillary','clinton','donald','trump','senate','house']

In [10]:
tagged_doc_pa = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(vocab_political_affiliation)]

In [12]:
from gensim.models.doc2vec import Doc2Vec
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_doc_pa)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_doc_pa,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model.pa")
print("Model Saved")   

Model Saved


In [13]:
df_train_statements_d2v=df_train[['statement','label']]
model= Doc2Vec.load("d2v.model.pa")
texts=[]
for x in df_train_statements_d2v['statement']:
    texts.append(model.infer_vector(x))

In [14]:
def replace_label(x):
    if (x == 'true' or x=='mostly-true' or x=='half-true'):
        return 1
    else:
        return 0
replace_label('true')    

1

In [15]:
X=pd.DataFrame(texts)
#y=df_train_statements_d2v[['label']]
y=df_train_statements_d2v['label'].map(lambda x:replace_label(x))
X_train,X_test,y_train,y_test=train_test_split(X, y,test_size = .3, random_state = 1)

In [16]:
from sklearn.linear_model import LogisticRegression
logisticRegr_D2V_PA = LogisticRegression(C=100)
logisticRegr_D2V_PA.fit(X_train, y_train)
lr_pred_pa = logisticRegr_D2V_PA.predict(X_test)
import pickle
s = pickle.dumps(logisticRegr_D2V_PA)

In [20]:
from sklearn import metrics
print(metrics.classification_report(y_test,lr_pred_pa))

             precision    recall  f1-score   support

          0       0.50      0.19      0.28      1331
          1       0.58      0.86      0.69      1741

avg / total       0.55      0.57      0.51      3072



In [21]:
import pickle
def political_affiliation_checker(news):
    data_pred=[]
    data_pred.append(model.infer_vector(news))
    lrg_pa = pickle.loads(s)
    pred_conf=lrg_pa.predict_proba(data_pred)
    #print(pred_conf)
    return pred_conf[0][1]

In [22]:
class PoliticalAffilicationDetector:
    def __init__(self,news):
        self.news=news
    def predict(self):
        return political_affiliation_checker(self.news)

In [23]:
political_affiliation_checker("Says the Annies List political group supports third-trimester abortions on demand.")

0.46289932276995244

In [24]:
logisticRegr_D2V_PA.classes_

array([0, 1])